In [1]:
import subprocess

try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])
    import geemap

In [2]:
import ee

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

In [3]:
from ipyleaflet import *
from ipywidgets import *
from geemap.utilities import *
Map = geemap.Map(layers=['Google Map'])
Map.default_style = {'cursor': 'pointer'}
Map

# Load National Hydrography Dataset (NHD) 
HUC10 = ee.FeatureCollection('USGS/WBD/2017/HUC10');  # 18,487 HUC10 watersheds in the U.S.

# Add HUC layer to the map
Map.setCenter(-99.00, 47.01, 8);
Map.addLayer(ee.Image().paint(HUC10, 0, 1), {}, 'HUC-10 Watershed')   # HUC10 for the entire U.S.

label = Label("Click on the map to select a watershed")
widget_control = WidgetControl(widget=label, position='bottomright')
Map.add_control(widget_control)

layer = None

def handle_interaction(**kwargs):
    latlon = kwargs.get('coordinates')
    if kwargs.get('type') == 'click':
        Map.default_style = {'cursor': 'wait'}
        xy = ee.Geometry.Point(latlon[::-1])
        watershed = HUC10.filterBounds(xy)
        huc10_id = watershed.first().get('huc10').getInfo()
        Map.layers = Map.layers[:3]
        Map.addLayer(ee.Image().paint(watershed, 0, 2), {'palette': 'red'}, 'HUC ID: ' + huc10_id) 
        
        NAIP_images = findNAIP(watershed)
        first_image = ee.Image(NAIP_images.toList(5).get(0))
        Map.addLayer(first_image, {'bands': ['N', 'R', 'G']}, 'first image')
#         count = NAIP_images.size().getInfo()
#         for i in range(0, count):
#             image = ee.Image(NAIP_images.toList(count).get(i))
#             Map.addLayer(image, {'bands': ['N', 'R', 'G']}, str(i))
        
        Map.default_style = {'cursor': 'pointer'}

Map.on_interaction(handle_interaction)

Map

Map(center=[47.01, -99.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_o…

In [4]:
Map.layers

(TileLayer(base=True, max_zoom=19, min_zoom=1, options=['attribution', 'detect_retina', 'max_native_zoom', 'max_zoom', 'min_native_zoom', 'min_zoom', 'no_wrap', 'tile_size', 'tms']),
 TileLayer(attribution='Google', name='Google Map', options=['attribution', 'detect_retina', 'max_native_zoom', 'max_zoom', 'min_native_zoom', 'min_zoom', 'no_wrap', 'tile_size', 'tms'], url='https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}'),
 TileLayer(attribution='Google Earth Engine', name='HUC-10 Watershed', options=['attribution', 'detect_retina', 'max_native_zoom', 'max_zoom', 'min_native_zoom', 'min_zoom', 'no_wrap', 'tile_size', 'tms'], url='https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/maps/89353059c6daabea7bd36154c9f5596f-23b0ebcb5129383766037cbc7b03501a/tiles/{z}/{x}/{y}'))